In [1]:
import ee
import geemap
import os

In [2]:
Map = geemap.Map()

In [4]:
countries = ee.FeatureCollection('users/giswqs/public/countries')
roi = countries.filter(ee.Filter.eq('id', 'CHL'))
Map.addLayer(roi, {}, 'roi')

In [5]:
start_date = '2020-12-01'
end_date = '2021-03-31'

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date)


In [6]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [8]:
Map.addLayer(l8, vizParams, 'false color composite')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [9]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [10]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [11]:
def addMonth(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('M'))
    return image.addBands(ee.Image(img_doy).rename('month').toInt())

In [12]:
def addDOY(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('D'))
    return image.addBands(ee.Image(img_doy).rename('doy').toInt())

In [13]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B3','B5']).rename('NDWI')
    return image.addBands(ndwi)

In [14]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B3','B4']).rename('NDGI')
    return image.addBands(ndgi)

In [15]:
coleccion = l8.map(addNDVI).map(addDate).map(addMonth).map(addDOY).map(addNDWI).map(addNDGI)

In [16]:
greenest = coleccion.qualityMosaic('NDVI')

In [20]:
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map.addLayer(greenest, visParams, 'comp mas verde')
Map.centerObject(roi, 6)
Map

Map(bottom=10971.0, center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['positio…

In [21]:
aquast = coleccion.qualityMosaic('NDWI')

In [22]:
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map.addLayer(aquast, visParams, 'comp mas agua')
Map.centerObject(roi, 6)
Map

Map(bottom=21467.0, center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['positio…

In [23]:
def Cloudcorte(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20))   

In [44]:
def aguacorte(image):
    return image.updateMask(aquast)  

In [45]:
sinagua = coleccion.map(Cloudcorte)

In [46]:
greenest_sinagua = sinagua.qualityMosaic('NDVI')

In [47]:
Map.addLayer(greenest_sinagua, visParams, 'comp sin agua')
Map.centerObject(roi, 6)
Map

Map(bottom=43309.0, center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['positio…

In [53]:
scored = ee.Algorithms.Landsat.simpleCloudScore(l8).select('cloud')

EEException: Landsat.simpleCloudScore, argument 'image': Invalid type.
Expected type: Image<unknown bands>.
Actual type: ImageCollection.

In [55]:
Map.addLayer(replaced, visParams, 'test mos')
Map.centerObject(roi, 6)
Map

EEException: Landsat.simpleCloudScore, argument 'image': Invalid type.
Expected type: Image<unknown bands>.
Actual type: ImageCollection.